In [2]:
import pandas as pd
import sqlite3
db=sqlite3.connect('sqlite3.db')

In [5]:
data=pd.read_sql('select * from get',db)

In [11]:
# data[data.duplicated(subset=['OI','Date','instr'],keep='first')].sort_values(['OI','Date','instr'])
data.drop_duplicates(subset=['OI','Date','instr'],keep='first',inplace=True)

In [18]:
data.Date=pd.to_datetime(data.Date)

In [49]:
data30ex=data[data.instr.str.startswith('NIFTY21SEP')]

In [50]:
data30ex['pece']=data.instr.str[-2:]

C:\Users\ssmgf\AppData\Local\Temp/ipykernel_12716/2247014453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data30ex['pece']=data.instr.str[-2:]


In [51]:
data30ex['strike']= data30ex.instr.str[-7:-2]

C:\Users\ssmgf\AppData\Local\Temp/ipykernel_12716/3598244868.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data30ex['strike']= data30ex.instr.str[-7:-2]


In [52]:
pd.pivot_table(data30ex,index='strike',columns='pece',values='file',aggfunc='count')


pece,CE,PE,UT
strike,,,
17700,456.0,456.0,NaN
17750,456.0,456.0,NaN
17800,456.0,456.0,NaN
17850,456.0,456.0,NaN
17900,456.0,456.0,NaN
17950,456.0,456.0,NaN
18000,456.0,456.0,NaN
18100,456.0,NaN,NaN
1SEPF,NaN,NaN,456.0


In [53]:
data30ex=data30ex[~data30ex.strike.isin(['1SEPF','18100'])]

In [54]:
data30ex.shape

(6384, 7)

In [56]:
data30ex[data30ex.duplicated(subset=['Date','instr'])]

,OI,Date,LTP,file,instr,strike,pece


In [57]:
data[data.duplicated(subset=['Date','instr'])]


,OI,Date,LTP,file,instr,strike


In [61]:
datapeceoi=data30ex.pivot_table(index='Date',columns='pece',values='OI',aggfunc='sum')

In [64]:
datapeceoi

pece,CE,PE
Date,,
2021-09-22 09:15:00+05:30,62561,29741
2021-09-22 09:20:00+05:30,63705,30095
2021-09-22 09:25:00+05:30,63953,30173
2021-09-22 09:30:00+05:30,64038,30100
2021-09-22 09:35:00+05:30,64240,30076
...,...,...
2021-09-27 15:10:00+05:30,225914,176602
2021-09-27 15:15:00+05:30,223849,175096
2021-09-27 15:20:00+05:30,220641,169947


In [65]:
datapeceoi['oidiff']=datapeceoi.PE-datapeceoi.CE

In [72]:
data.set_index('Date',drop=True,inplace=True)

KeyError: "None of ['Date'] are in the columns"

In [75]:
datapeceoi= datapeceoi.join( data[data.instr=='NIFTY']['LTP'])


In [77]:
import cufflinks
from plotly.offline import iplot, init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [79]:
fig=datapeceoi.iplot(y=['oidiff'],secondary_y=['LTP'],asFigure=True)
fig.write_html('check_result.html')